# Evaluating Home Field Advantage During the 2018 Regular NFL Season

## Import needed packages and config

In [5]:
import requests
import mysql.connector
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Connect to MySql  and create database. Commented out once DB is created

In [6]:
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
)
cursor = cnx.cursor(buffered=True)

## (re)Connect to database, with database defined, to create table

In [23]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'sports'
)
cursor = cnx.cursor(buffered=True)

## Make API Call to grab 2019 MLB reg season data


In [8]:
api_key = config.mlb_key
headers = {'Ocp-Apim-Subscription-Key' : '{}'.format(api_key)}

In [9]:
mlb_data = []
url = 'https://api.sportsdata.io/v3/mlb/scores/json/Games/2019'
response = requests.get(url, headers=headers)
data = response.json()
mlb_data.append(data)

In [10]:
mlb_data = mlb_data[0] # data came back as list of lists of dictionaries.

In [24]:
mlb_data

[{'GameID': 54052,
  'Season': 2019,
  'SeasonType': 1,
  'Status': 'Final',
  'Day': '2019-03-20T00:00:00',
  'DateTime': '2019-03-20T05:35:00',
  'AwayTeam': 'SEA',
  'HomeTeam': 'OAK',
  'AwayTeamID': 13,
  'HomeTeamID': 24,
  'RescheduledGameID': None,
  'StadiumID': 69,
  'Channel': 'ESPN',
  'Inning': 9,
  'InningHalf': 'B',
  'AwayTeamRuns': 14,
  'HomeTeamRuns': 11,
  'AwayTeamHits': 11,
  'HomeTeamHits': 14,
  'AwayTeamErrors': 3,
  'HomeTeamErrors': 0,
  'WinningPitcherID': 10003325,
  'LosingPitcherID': 10000694,
  'SavingPitcherID': 10001121,
  'Attendance': None,
  'AwayTeamProbablePitcherID': 10003325,
  'HomeTeamProbablePitcherID': 10000694,
  'Outs': None,
  'Balls': None,
  'Strikes': None,
  'CurrentPitcherID': None,
  'CurrentHitterID': None,
  'AwayTeamStartingPitcherID': 10003325,
  'HomeTeamStartingPitcherID': 10000694,
  'CurrentPitchingTeamID': None,
  'CurrentHittingTeamID': None,
  'PointSpread': -2.4,
  'OverUnder': 14.4,
  'AwayTeamMoneyLine': 231,
  'HomeTe

In [12]:
parsed_games_mlb = []
for game in mlb_data:
    parsed_game = {}
    parsed_game['game_id'] = game['GameID']
    parsed_game['month'] = game['DateTime'][5:7]
    parsed_game['away_team'] = game['AwayTeam']
    parsed_game['away_team_score'] = game['AwayTeamRuns']
    parsed_game['away_team_moneyline'] = game['PointSpreadAwayTeamMoneyLine']
    parsed_game['home_team'] = game['HomeTeam']
    parsed_game['home_team_score'] = game['HomeTeamRuns']
    parsed_game['home_team_moneyline'] = game['PointSpreadHomeTeamMoneyLine']
    parsed_game['spread'] = game['PointSpread']
    parsed_games_mlb.append(parsed_game)

In [25]:
parsed_games_mlb

[{'game_id': 54052,
  'month': '03',
  'away_team': 'SEA',
  'away_team_score': 14,
  'away_team_moneyline': -263,
  'home_team': 'OAK',
  'home_team_score': 11,
  'home_team_moneyline': 237,
  'spread': -2.4},
 {'game_id': 54066,
  'month': '03',
  'away_team': 'SEA',
  'away_team_score': 13,
  'away_team_moneyline': -372,
  'home_team': 'OAK',
  'home_team_score': 10,
  'home_team_moneyline': 332,
  'spread': -2.4},
 {'game_id': 54155,
  'month': '03',
  'away_team': 'BAL',
  'away_team_score': 5,
  'away_team_moneyline': 258,
  'home_team': 'NYY',
  'home_team_score': 11,
  'home_team_moneyline': -301,
  'spread': -2.4},
 {'game_id': 54156,
  'month': '03',
  'away_team': 'NYM',
  'away_team_score': 5,
  'away_team_moneyline': -330,
  'home_team': 'WSH',
  'home_team_score': 0,
  'home_team_moneyline': 295,
  'spread': -2.4},
 {'game_id': 54157,
  'month': '03',
  'away_team': 'STL',
  'away_team_score': 10,
  'away_team_moneyline': -305,
  'home_team': 'MIL',
  'home_team_score': 1

## Inserting to table

In [26]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'sports'
)
cursor = cnx.cursor(buffered=True)

In [14]:
# cursor.execute("""
# CREATE DATABASE MLB
# """)

# cursor.close()
# cnx.close()

In [27]:
cursor.execute("""
    CREATE TABLE MLB_2019_GAMES (
    game_id varchar(11) NOT NULL primary key,
    month varchar(2) not null,
    away_team varchar(11) not null,
    away_team_score int(3) not null,
    home_team varchar(11) not null,
    home_team_score int(3) not null,
    away_team_moneyline int(4) not null,
    home_team_moneyline int(4) not null,
    spread float(5) not null
    )
""")

In [28]:
mlb_games_to_insert = []

for game in parsed_games_mlb:

    mlb_game_to_insert = (
        game['game_id'], 
        game['month'], 
        game['away_team'],
        game['away_team_score'],
        game['home_team'], 
        game['home_team_score'],
        game['away_team_moneyline'],
        game['home_team_moneyline'],
        game['spread'] 
    )
    
    mlb_games_to_insert.append(mlb_game_to_insert)

In [29]:
insert_query = """
    INSERT INTO MLB_2019_GAMES (
            game_id,
            month,
            away_team,
            away_team_score,
            home_team,
            home_team_score,
            away_team_moneyline,
            home_team_moneyline,
            spread
        )
    VALUES (
            %s,
            %s,
            %s,
            %s,
            %s,
            %s,
            %s,
            %s,
            %s
        )
"""
cursor.executemany(insert_query, mlb_games_to_insert)
cnx.commit()

### Questions (Null Hypothesis)

Is there a "home field" advantage(in general , 2018 season)?
    
Will a home team preform better (by month)?

If home team cover more than away team ("good teams win, great teams cover")?


## Creating MLB DataFrame

In [30]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = 'sports'
)
cursor = cnx.cursor(buffered=True)

In [33]:
cursor.execute(  """SELECT * FROM MLB_2019_GAMES
                    order by game_id;""")
#assigning database (pointed in MySql) as a dataframe using pandas                         
df_mlb = pd.DataFrame(cursor.fetchall())


#assigning the column names 
df_mlb.columns=['game_id','month', 'away_team', 'away_team_score', 'home_team', 'home_team_score', 'away_team_moneyline', 'home_team_moneyline', 'spread']

In [34]:
df_mlb

,game_id,month,away_team,away_team_score,home_team,home_team_score,away_team_moneyline,home_team_moneyline,spread
0,54052,03,SEA,14,OAK,11,-263,237,-2.4
1,54066,03,SEA,13,OAK,10,-372,332,-2.4
2,54152,03,CHW,8,KC,13,-306,274,-2.4
3,54153,03,CHC,19,TEX,10,205,-240,3.9
4,54154,03,HOU,13,TB,3,213,-260,3.9
5,54155,03,BAL,5,NYY,11,258,-301,-2.4
6,54156,03,NYM,5,WSH,0,-330,295,-2.4
7,54157,03,STL,10,MIL,13,-305,274,-2.4
8,54158,03,ATL,10,PHI,16,-208,176,-2.4
9,54159,03,DET,5,TOR,0,-244,213,-2.4
